# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,3.34,94,100,2.40,GS,1684538871
1,1,hamilton,39.1834,-84.5333,24.58,68,20,6.69,US,1684538811
2,2,kone,-21.0595,164.8658,25.00,47,34,2.61,NC,1684539042
3,3,fale old settlement,-9.3852,-171.2468,28.61,81,58,6.04,TK,1684538942
4,4,sandwip,22.4678,91.4561,28.60,83,1,5.91,BD,1684539043


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
# %%capture --no-display

# Configure the map plot using hvplot
city_humidity_map = city_data_df.hvplot.points("Lng", 
                                               "Lat", 
                                               geo = True, 
                                               tiles = "OSM", 
                                               frame_width = 800, 
                                               frame_height = 600, 
                                               size = "Humidity", 
                                               scale = 0.8, 
                                               color = "City", 
                                               hover_cols = ["City"])

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [65]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df[(city_data_df["Max Temp"] < 28) & (city_data_df["Max Temp"] > 18)
                                    & (city_data_df["Cloudiness"] >= 30) & (city_data_df["Wind Speed"] >= 3)] 

# Drop any rows with null values
ideal_weather_cities.dropna()

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,carnarvon,-24.8667,113.6333,19.75,38,41,9.05,AU,1684538991
23,23,roma,43.2128,-75.4557,22.08,46,100,4.12,US,1684539048
25,25,vieux fort,13.7167,-60.9500,27.62,80,40,10.29,LC,1684539048
34,34,avarua,-21.2078,-159.7750,26.03,57,81,3.09,CK,1684539050
41,41,west island,-12.1568,96.8225,24.99,94,100,6.17,CC,1684539052
...,...,...,...,...,...,...,...,...,...,...
536,536,muntok,-2.0667,105.1833,25.83,89,67,3.15,ID,1684539186
544,544,ceelbuur,4.6850,46.6176,26.03,73,86,5.81,SO,1684539188
558,558,isangel,-19.5500,169.2667,27.06,74,75,3.60,VU,1684539192
561,561,shindand,33.3029,62.1474,23.23,19,71,3.01,AF,1684539193


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
17,carnarvon,AU,-24.8667,113.6333,38,
23,roma,US,43.2128,-75.4557,46,
25,vieux fort,LC,13.7167,-60.9500,80,
34,avarua,CK,-21.2078,-159.7750,57,
41,west island,CC,-12.1568,96.8225,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {
    "categories" : categories,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Define a counter to keep track of the number of cities processed
city_count = 0

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Check if the desired number of cities have been processed (per instructions, trying to limit to 10 or so with hotels)
    if city_count >= 15:
        break
    
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    # Increment the city_count by 1 
    city_count += 1

# Display sample data
hotel_df.head()

Starting hotel search
carnarvon - nearest hotel: No hotel found
roma - nearest hotel: Wingate by Wyndham Rome
vieux fort - nearest hotel: Kimatrai Hotel
avarua - nearest hotel: Paradise Inn
west island - nearest hotel: Cocos Village Bungalows
suicheng - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
hawaiian paradise park - nearest hotel: No hotel found
al qubbah - nearest hotel: No hotel found
flying fish cove - nearest hotel: Christmas Island Lodge
praia da vitoria - nearest hotel: Salles
touros - nearest hotel: Pousada Atlântico
puerto baquerizo moreno - nearest hotel: Dory's House
bandarbeyla - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
17,carnarvon,AU,-24.8667,113.6333,38,No hotel found
23,roma,US,43.2128,-75.4557,46,Wingate by Wyndham Rome
25,vieux fort,LC,13.7167,-60.9500,80,Kimatrai Hotel
34,avarua,CK,-21.2078,-159.7750,57,Paradise Inn
41,west island,CC,-12.1568,96.8225,94,Cocos Village Bungalows


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
# %%capture --no-display

# Configure the map plot
ideal_cities_map = hotel_df.hvplot.points("Lng", 
                                          "Lat", 
                                          geo = True, 
                                          tiles = "OSM", 
                                          frame_width = 800, 
                                          frame_height = 600, 
                                          size = "Humidity", 
                                          scale = 0.8, 
                                          color = "City", 
                                          hover_cols = ["City", "Hotel Name", "Country"])

# Display the map
ideal_cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)